# Mini Intro to Textual Analysis: Wordlist
by Dr Liang Jin

Part of Mini Python Sessions: [github.com/drliangjin/minipy](https://github.com/drliangjin/minipy)

Bodnaruk, Loughran, and McDonald (2015)

Note: Predefined SEC strings
- 10K filing codes: '10-K', '10-K405', '10KSB', '10-KSB', '10KSB40'
- 10Q filing codes: '10-Q', '10QSB', '10-QSB'

Excellent (but outdated) resources by Bill McDonald can be found from [Software Repository for Accounting and Finance](https://sraf.nd.edu/). It's relevant but dangerous to use those codes straightway.

In [1]:
# NLTK stands for Natural Language Took Kits
# It is the most popular advanced textual analysis tool/package in Python
# It has tons of features and also comes with a large collection of corpus (a collection of text) to play with
# These features however need to be downloaded by running the following:
import nltk
#nltk.download(); #This thing is huge, please note it will take quite sometime to finalise the downloads.

## Getting Text Data

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
# Apple's 10-K filing on 2017
# IBM's 10-K filing on 20120228
urls = ['https://www.sec.gov/Archives/edgar/data/51143/000104746912001742/a2206744z10-k.htm',
        'https://www.sec.gov/Archives/edgar/data/320193/000032019317000070/a10-k20179302017.htm']

### Soup, HTML and Text
- Remove tables: All characters appearing between `<TABLE>` and `</TABLE>` tages are removed 
- NOTE: unless numeric characters/(alphabetic + numeric chars) <= 15% (BLM, 2015), **can you do this?**

In [4]:
# Define a function to create our Soup object and then extract text
# The key is here is: when we have HTML structure, we remove tables otherwise it can be tricky
def url_to_text(url):
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')
    for table in soup.find_all('table'):
        table.decompose()
    text = soup.get_text()
    return text

In [5]:
# Actually obtain the text from requests via EDGAR, parsing using BS4 then text
texts = [url_to_text(url) for url in urls]

In [6]:
# have a peek!
texts[0]


'\n10-K\n1\na2206744z10-k.htm\n10-K\n\n\n\n\n\n\nQuickLinks\n -- Click here to rapidly navigate through this document\n\n\n\n\n\n\n\n\n \n\n \nUNITED STATES\nSECURITIES AND EXCHANGE COMMISSION WASHINGTON, D.C. 20549  \n\n\n\n\n\n \nFORM 10-K ANNUAL REPORT\npursuant to Section\xa013 or 15 (d)\xa0of the\nSecurities Exchange Act of 1934\nFOR THE YEAR ENDED DECEMBER 31, 2011  \n 1-2360 (Commission file number) \nINTERNATIONAL BUSINESS MACHINES CORPORATION (Exact name of registrant as specified in its charter) \n\n\n\n\n\n\n\n\n914-499-1900 (Registrant\'s telephone number) \nSecurities registered pursuant to Section\xa012(b) of the Act:  \n\n\n\n\n\n\n\n\n\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Indicate by check mark if the registrant is a well-known seasoned issuer as defined in Rule\xa0405 of the Securities Act.\nYes\xa0ý\xa0\xa0\xa0\xa0No\xa0o \n\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Indicate by check mark if the registrant is not required to file reports pursuant to Section\xa013 or Section\xa

### Store our data within Python

In [7]:
# hold files locally
import pickle # nice module name, isn't it?

firms = ['Apple', 'IBM']

# access index and value for a list
for idx, val in enumerate(firms):
    with open(val + ".pkl", "wb") as f:
        pickle.dump(texts[idx], f)

### Load our pickled data into a dictionary

In [8]:
# Load pickled files into a dictionary
# Key: company name
# value: parsed 10K text
data = {}

for _, val in enumerate(firms):
    with open(val + ".pkl", "rb") as f:
        data[val] = pickle.load(f)

In [9]:
# Check company names to make sure our data has been loaded properly
data.keys()

dict_keys(['Apple', 'IBM'])

In [10]:
# Check texts
#
data['IBM'][:2000]

'\n10-K\n1\na10-k20179302017.htm\n10-K\n\n\n\n\n\n\nDocument\n\n\nUNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549FORM 10-K(Mark One)☒ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the fiscal year ended September\xa030, 2017or☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the transition period from\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 to \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Commission File Number: 001-36743Apple Inc.(Exact name of Registrant as specified in its charter)(408) 996-1010(Registrant’s telephone number, including area code)Securities registered pursuant to Section\xa012(b)\xa0of the Act:Securities registered pursuant to Section\xa012(g)\xa0of the Act:  NoneIndicate by check mark if the Registrant is a well-known seasoned issuer, as defined in Rule 405 of the Securities Act.Yes\xa0\xa0☒\xa0\xa0\xa0\xa0No\xa0\xa0☐Indicate by check mark if the 

## (very) Basic Features Extraction

In [11]:
# Let's first define some handy funcs
# Count total Words
def count_words(text):
    return len(str(text).split(" "))

# Count total characters
def count_chars(text):
    return len(str(text))

# Count numerics
def count_digit(text):
    return len([word for word in str(text).split(" ") if word.isdigit()]) # isdigit() is a string method

In [12]:
# Stopwords are meaningless...
# NOTE: you need to run `nltk.download()` and download required nltk packages to access this feature
from nltk.corpus import stopwords

In [13]:
# Count stopwords
def count_stopwords(text, stop=stopwords.words('english')):
    num = len([word for word in str(text).split(' ') if word in stop])
    perc = num/len(str(text).split(" "))
    return num, perc

In [14]:
# stopwords list from LM's website

lm_list = ['ME', 'MY', 'MYSELF', 'WE', 'OUR', 'OURS', 'OURSELVES', 'YOU', 'YOUR', 'YOURS',
                       'YOURSELF', 'YOURSELVES', 'HE', 'HIM', 'HIS', 'HIMSELF', 'SHE', 'HER', 'HERS', 'HERSELF',
                       'IT', 'ITS', 'ITSELF', 'THEY', 'THEM', 'THEIR', 'THEIRS', 'THEMSELVES', 'WHAT', 'WHICH',
                       'WHO', 'WHOM', 'THIS', 'THAT', 'THESE', 'THOSE', 'AM', 'IS', 'ARE', 'WAS', 'WERE', 'BE',
                       'BEEN', 'BEING', 'HAVE', 'HAS', 'HAD', 'HAVING', 'DO', 'DOES', 'DID', 'DOING', 'AN',
                       'THE', 'AND', 'BUT', 'IF', 'OR', 'BECAUSE', 'AS', 'UNTIL', 'WHILE', 'OF', 'AT', 'BY',
                       'FOR', 'WITH', 'ABOUT', 'BETWEEN', 'INTO', 'THROUGH', 'DURING', 'BEFORE',
                       'AFTER', 'ABOVE', 'BELOW', 'TO', 'FROM', 'UP', 'DOWN', 'IN', 'OUT', 'ON', 'OFF', 'OVER',
                       'UNDER', 'AGAIN', 'FURTHER', 'THEN', 'ONCE', 'HERE', 'THERE', 'WHEN', 'WHERE', 'WHY',
                       'HOW', 'ALL', 'ANY', 'BOTH', 'EACH', 'FEW', 'MORE', 'MOST', 'OTHER', 'SOME', 'SUCH',
                       'NO', 'NOR', 'NOT', 'ONLY', 'OWN', 'SAME', 'SO', 'THAN', 'TOO', 'VERY', 'CAN',
                       'JUST', 'SHOULD', 'NOW']

lm_stopwords = [word.lower() for word in lm_list]

In [15]:
# A very large proportion of the whole text is stopwords!
count_stopwords(data['IBM'])

(12611, 0.34762114780307624)

In [16]:
# or try out LM stopwords
count_stopwords(data['IBM'], stop=lm_stopwords)

(12173, 0.33554771486851537)

### Load our dictionary into Pandas DataFrame

In [17]:
# We can either keep it in dictionary format or put it into a pandas dataframe
# put our corpus into a pandas dataframe
import pandas as pd
pd.set_option('max_colwidth', 150)

df = pd.DataFrame.from_dict(data, orient='index', columns = ['text'])

In [18]:
df

,text
Apple,\n10-K\n1\na2206744z10-k.htm\n10-K\n\n\n\n\n\n\nQuickLinks\n -- Click here to rapidly navigate through this document\n\n\n\n\n\n\n\n\n \n\n \nUNIT...
IBM,"\n10-K\n1\na10-k20179302017.htm\n10-K\n\n\n\n\n\n\nDocument\n\n\nUNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549FORM 10-K(Ma..."


## Cleaning the Text Data (Text Pre-Processing)

Data analysis is more of art than science, in a sense. For example, when we are trying to clean our data, we need manual inputs and our judgements. After all, no data can be perfect; especially for text data, the cleaning or pre-processing can go on forever. We are just going to execute the most common/simple cleaning steps; you can continue to work on to improve your results, i.e., replicating BLM(2015)'s work...

### Common data cleanning steps:
- Make text all lower case
- Remove special expression from non-human languages
- Remove punctuation
- Remove numerical values
- Remove stop words
- Tokenize text

### More advanced cleaning steps after tokenization:
- Stemming / lemmatization
- Tagging
- N-grams
- And more...

### Replicate BLM(2015) on constraining words
- Read the paper carefully, focusing on sections such as `II. Data`
- Go through `Appendix B. Parsing the 10-K Filings` (discard the first 4 steps for now as they are for txt files)

In [19]:
# OK, basic text cleaning
import re


In [20]:
# deal with reserved special html characters such as non-breaking space (`&nbsp`)
html_chars = {'&lt': 'lt', '&#60': 'lt', 
              '&gt': 'gt', '&#62': 'gt',
              '&nbsp': '', '&#160': '', 
              '&quot': '"', '&#34': '"', 
              '&apos': '\'', '&#39': '\'',
              '&amp': '&', '&#38': '&'}

In [21]:
def clean_text_round1(text):
    pass




In [22]:
def clean_text_round2(text):
    # convert to lower case
    text = text.lower()
    text = re.sub(r'(\t|\v)', '', text)
    # remove \xa0 which is non-breaking space from ISO 8859-1, how to delete all remaining ISO 8859-1 symbols & chars?
    text = re.sub(r'\xa0', ' ', text)
    # remove newline feeds (\n) following hyphens
    text = re.sub(r'(-+)\n{2,}', r'\1', text)
    # remove hyphens preceded and followed by a blank space
    text = re.sub(r'\s-\s', '', text)
    # replace 'and/or' with 'and or'
    text = re.sub(r'and/or', r'and or', text)
    # tow or more hypens, periods, or equal signs, possiblly followed by spaces are removed
    text = re.sub(r'[-|\.|=]{2,}\s*', r'', text)
    # all underscores are removed
    text = re.sub(r'_', '', text)
    # 3 or more spaces are replaced by a single space
    text = re.sub(r'\s{3,}', ' ', text)
    # three or more line feeds, possibly separated by spaces are replaced by two line feeds
    text = re.sub(r'(\n\s*){3,}', '\n\n', text)
    # remove hyphens before a line feed
    text = re.sub(r'-+\n', '\n', text)
    # replace hyphens preceding a capitalized letter with a space
    text = re.sub(r'-+([A-Z].*)', r' \1', text)
    # remove capitalized or all capitals for March, May and August
    text = re.sub(r'(March|MARCH|May|MAY|August|AUGUST)', '', text)
    # remove punctuations
    # text = re.sub('[]'.format(re.escape(string.punctuation)), '', text)
    # remove line feeds
    # text = re.sub('\n', ' ', text)
    # remove numbers?
    # replace single line feed \n with single space
    #text = re.sub(r'\n', ' ', text)
    return text

# NOTE: 
## drop punctuation within numbers for number count
    #text = re.sub('(?!=[0-9])(\.|,)(?=[0-9])', '', text)

In [23]:
df

,text
Apple,\n10-K\n1\na2206744z10-k.htm\n10-K\n\n\n\n\n\n\nQuickLinks\n -- Click here to rapidly navigate through this document\n\n\n\n\n\n\n\n\n \n\n \nUNIT...
IBM,"\n10-K\n1\na10-k20179302017.htm\n10-K\n\n\n\n\n\n\nDocument\n\n\nUNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549FORM 10-K(Ma..."


In [24]:
df1 = pd.DataFrame(df['text'].apply(clean_text_round2))

In [25]:
df1

,text
Apple,\n10-k\n1\na2206744z10-k.htm\n10-k quicklinks\n click here to rapidly navigate through this document united states\nsecurities and exchange commis...
IBM,"\n10-k\n1\na10-k20179302017.htm\n10-k document united statessecurities and exchange commissionwashington, d.c. 20549form 10-k(mark one)☒ annual re..."


## Advanced Text Processing

### Document-Term Matrix

To continue working on our textual analysis of 10-K filings (can be as simple as word counts or can be as fancy as machine learning based techniques, the text must be tokenized, meaning broken down into smaller pieces. NLTK provides methods to do so, such as breaking text into sentenses and words. We can also do this using scikit-learn's CountVectorizer. The output will be multiple rows representing different documents (such a 10-K file) and multiple columns (lots of columns) representing a different word.

In [ ]:
# We are going to create a document-term matrix using CountVectorizer
# NOTE: we can remove stop words which are common words that add no additional meaning to the text, such as 'a', 'the', etc.
# NOTE: later we can try use LM defined stopwords for 10-K, we can even create our own stopwords dictionary
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
cv_data = cv.fit_transform(df.text)

dtm = pd.DataFrame(cv_data.toarray(), columns=cv.get_feature_names())
dtm.index = df.index
dtm

In [ ]:
# Let's pickle our dataframes
dtm.to_pickle('dtm.pkl')

# and our CountVectorizer object
with open("cv.pkl", "wb") as f:
    pickle.dump(cv, f)

## Exploratory Data Analysis

### Top words

In [ ]:
data = dtm.transpose()
data.head(50)



In [ ]:
# Find the most common words used by the 10-K files
top_words = {}

for firm in data.columns:
    top = data[firm].sort_values(ascending=False).head(30)
    top_words[firm] = list(zip(top.index, top.values))
    
    

In [ ]:
top_words


In [ ]:
# Print top words for each company
for firm, words in top_words.items():
    print(firm)
    print(','.join([word for word, count in words[0:14]]))
    print('------')
    
    

In [ ]:
# Create a bag of contrainning words from BLM (2015)
constraining_words = []

with open('words_from_pdf.txt', 'r') as rf:
        lines = rf.read().splitlines() # readlines() create a newline character "\n" each line
        for line in lines:
            words = line.split(sep=' ')
            for word in words:
                constraining_words.append(word)

# sort words alphabetically               
constraining_words.sort()

# You can write to a local file of course    

In [ ]:
constraining_words



In [23]:
import pandas as pd 
df=pd.read_excel("整合的.xlsx",sheet_name=2,skiprows=[1,2],headers=0,index_col=0)
df

,fileName,cik,date,companyName,n_count,n_ratio,total _word,c_count,c_ratio,filing_date,report_date,url
0,0000048681_2001-03-22.html,48681,2001-03-22,NaN,265,0.009071,29215.0,71,0.002430,2001-03-22,2000-12-31,https://www.sec.gov/Archives/edgar/data/48681/...
1,0001074271_2001-04-02.html,1074271,2001-04-02,NaN,15,0.003898,3848.0,7,0.001819,2001-04-02,2001-03-30,https://www.sec.gov/Archives/edgar/data/107427...
2,0000863779_2001-04-02.html,863779,2001-04-02,NaN,117,0.006312,18536.0,42,0.002266,2001-04-02,2000-12-31,https://www.sec.gov/Archives/edgar/data/863779...
3,0000011027_2001-03-30.html,11027,2001-03-30,NaN,107,0.009020,11862.0,60,0.005058,2001-03-30,2000-12-31,https://www.sec.gov/Archives/edgar/data/11027/...
4,0000877834_2001-03-30.html,877834,2001-03-30,NaN,245,0.007556,32425.0,164,0.005058,2001-03-30,2000-12-31,https://www.sec.gov/Archives/edgar/data/877834...
...,...,...,...,...,...,...,...,...,...,...,...,...
605,0000789932_2001-12-19.html,789932,2001-12-19,NaN,26,0.005567,4670.0,8,0.001713,2001-12-19,1998-08-31,https://www.sec.gov/Archives/edgar/data/789932...
606,0000884269_2001-03-19.html,884269,2001-03-19,NaN,34,0.002893,11754.0,27,0.002297,2001-03-19,2000-12-31,https://www.sec.gov/Archives/edgar/data/884269...
607,0000847431_2001-04-02.html,847431,2001-04-02,NaN,363,0.014012,25906.0,69,0.002663,2001-04-02,2000-12-31,https://www.sec.gov/Archives/edgar/data/847431...
608,0001130827_2001-03-29.html,1130827,2001-03-29,NaN,3,0.003492,859.0,9,0.010477,2001-03-29,2001-03-29,https://www.sec.gov/Archives/edgar/data/113082...


In [24]:
df.sort_values(by="c_count",ascending=False)

,fileName,cik,date,companyName,n_count,n_ratio,total _word,c_count,c_ratio,filing_date,report_date,url
231,0000750561_2001-04-02.html,750561,2001-04-02,NaN,784,0.010950,71595.0,417,0.005824,2001-04-02,2000-12-31,https://www.sec.gov/Archives/edgar/data/750561...
521,0000867665_2001-04-02.html,867665,2001-04-02,NaN,619,0.012685,48798.0,316,0.006476,2001-04-02,2000-12-31,https://www.sec.gov/Archives/edgar/data/867665...
86,0001043769_2001-05-24.html,1043769,2001-05-24,NaN,1108,0.019534,56722.0,306,0.005395,2001-05-24,2000-12-31,https://www.sec.gov/Archives/edgar/data/104376...
150,0000023426_2001-03-28.html,23426,2001-03-28,NaN,547,0.007868,69523.0,286,0.004114,2001-03-28,2000-12-31,https://www.sec.gov/Archives/edgar/data/23426/...
151,0000023426_2001-03-28.html,23426,2001-03-28,NaN,547,0.007868,69523.0,286,0.004114,2001-03-28,2000-12-31,https://www.sec.gov/Archives/edgar/data/23426/...
...,...,...,...,...,...,...,...,...,...,...,...,...
180,0000934843_2001-03-30.html,934843,2001-03-30,NaN,2,0.002581,775.0,1,0.001290,2001-03-30,2000-12-31,https://www.sec.gov/Archives/edgar/data/934843...
437,0001108022_2001-07-06.html,1108022,2001-07-06,NaN,1,0.001527,655.0,1,0.001527,2001-07-06,2000-12-31,https://www.sec.gov/Archives/edgar/data/110802...
310,0000930743_2001-04-17.html,930743,2001-04-17,NaN,2,0.001273,1571.0,1,0.000637,2001-04-17,2000-12-31,https://www.sec.gov/Archives/edgar/data/930743...
252,0001129612_2001-03-28.html,1129612,2001-03-28,NaN,2,0.003759,532.0,1,0.001880,2001-03-28,2000-12-31,https://www.sec.gov/Archives/edgar/data/112961...


In [1]:
import pandas as pd
for i in range(2,19):
    df=pd.read_excel("整合的.xlsx",sheet_name=i,skiprows=[1,2],headers=0,index_col=0)
    df=df.sort_values(by="c_count",ascending=False)
    if i ==2:
        df[["fileName","cik","filing_date"]].head(15).to_csv("data_1.csv")
    else:
        df[["fileName","cik","filing_date"]].head(15).to_csv("data_1.csv",header=None,mode="a")

In [44]:
df=pd.read_csv("data_1.csv",encoding="latin1",index_col=0)
#df["fileName"]
data=[]
fyear=[]
for i in range(len(df)):
    data.append(df["fileName"].values[i][:10])
    fyear.append(df["filing_date"].values[i])

In [46]:
fyear

['2001-04-02',
 '2001-04-02',
 '2001-05-24',
 '2001-03-28',
 '2001-03-28',
 '2001-03-28',
 '2001-03-28',
 '2001-03-28',
 '2001-09-28',
 '2001-04-02',
 '2001-04-02',
 '2001-04-02',
 '2001-03-23',
 '2001-10-11',
 '2001-04-02',
 '2002-04-01',
 '2002-04-01',
 '2002-04-01',
 '2002-03-27',
 '2002-12-16',
 '2002-03-25',
 '2002-03-25',
 '2002-03-01',
 '2002-03-05',
 '2002-12-27',
 '2002-05-31',
 '2002-10-09',
 '2002-04-01',
 '2002-07-05',
 '2002-03-18',
 '2003-04-04',
 '2003-03-28',
 '2003-04-23',
 '2003-12-16',
 '2003-03-28',
 '2003-03-28',
 '2003-03-31',
 '2003-03-10',
 '2003-01-14',
 '2003-08-12',
 '2003-03-12',
 '2003-05-01',
 '2003-03-28',
 '2003-03-31',
 '2003-07-01',
 '2004-03-15',
 '2004-03-11',
 '2004-02-27',
 '2004-04-19',
 '2004-03-01',
 '2004-03-15',
 '2004-03-15',
 '2004-03-15',
 '2004-04-14',
 '2004-03-05',
 '2004-03-09',
 '2004-03-09',
 '2004-03-09',
 '2004-03-09',
 '2004-03-09',
 '2005-03-31',
 '2005-05-02',
 '2005-03-31',
 '2005-03-23',
 '2005-03-15',
 '2005-03-10',
 '2005-03-